In [28]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import numpy as np
!pip install xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance
import xgboost as xgb



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [18]:
mass_balance_hy_df = pd.read_csv('project-glaciers/data/mass_balance_hy.csv')
silvrettahütte_prec = pd.read_csv('project-glaciers/data/weather_data_silvrettahütte.csv')
weissfluhjoch_temp = pd.read_csv('project-glaciers/data/weather_data_weissfluhjoch.csv')
klosters_prec = pd.read_csv('project-glaciers/data/weather_data_klosters.csv')
davos_t_dev = pd.read_csv('project-glaciers/data/weather_dev_davos_temp.csv')
davos_w = pd.read_csv('project-glaciers/data/weather_data_davos_monthly.csv')
davos_prec = davos_w[['rhs150m0', 
                          'date'
                     ]]
davos_temp = davos_w[['ths200m0',
                          'date'
                     ]]

davos_temp = davos_temp.set_index('date')
davos_temp = davos_temp[
    (davos_temp.index >= '1959-10-01') &
    (davos_temp.index < '2025-10-01')
]
weissfluhjoch_temp = weissfluhjoch_temp.set_index('date')
weissfluhjoch_temp = weissfluhjoch_temp[
    (weissfluhjoch_temp.index >= '1959-10-01') &
    (weissfluhjoch_temp.index < '2025-10-01')
]


silvrettahütte_prec = silvrettahütte_prec.set_index('date')
silvrettahütte_prec = silvrettahütte_prec[
    (silvrettahütte_prec.index >= '1959-01-01') &
    (silvrettahütte_prec.index < '2025-10-01')
]

klosters_prec = klosters_prec.set_index('date')
klosters_prec = klosters_prec[
    (klosters_prec.index >= '1959-10-01') &
    (klosters_prec.index < '2025-10-01')
]



weissfluhjoch_temp.rename(columns = {'mean temperature (°C)': 'WFJ mean temperature (°C)'}, inplace = True)
davos_temp.rename(columns = {'ths200m0': 'DAV mean temperature (°C)'}, inplace = True)


silvrettahütte_prec = silvrettahütte_prec.reset_index(drop = True)
klosters_prec = klosters_prec.reset_index(drop = True)
davos_t_dev



,october_td,november_td,december_td,january_td,february_td,march_td,april_td,may_td,june_td,july_td,august_td,september_td,hydrological year
0,-1.9,-2.6,0.2,-2.4,-1.2,-2.4,-2.4,1.3,-0.1,-2.5,-3.6,-2.5,1914-1915
1,-4.0,-4.2,2.3,0.1,-0.3,-0.4,-1.2,-0.7,-3.9,-2.7,-2.8,-3.2,1915-1916
2,-2.4,-1.7,-0.3,-3.9,-3.2,-3.1,-5.0,1.1,0.1,-1.8,-2.5,1.0,1916-1917
3,-4.5,-3.2,-5.1,-0.8,-1.1,-1.4,-0.5,0.1,-4.3,-2.2,-2.8,-0.4,1917-1918
4,-2.8,-1.2,0.2,-2.2,-2.4,-1.9,-3.8,-3.1,-2.1,-5.0,-0.6,0.4,1918-1919
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-1.4,2.1,0.3,-1.9,2.6,-0.9,-2.1,-2.4,2.2,-0.3,-1.1,1.8,2020-2021
107,-0.4,-1.0,0.6,0.6,0.8,0.7,0.3,2.3,2.5,2.0,1.7,-0.6,2021-2022
108,3.5,1.3,1.1,0.4,2.0,1.6,-0.9,0.5,2.0,1.2,1.0,3.7,2022-2023
109,3.1,-1.1,1.4,1.2,4.9,2.2,0.6,0.4,0.7,1.8,2.9,-0.3,2023-2024


In [3]:
silvretta_temp = weissfluhjoch_temp.merge(davos_temp, left_index=True, right_index=True)
silvretta_temp['difference/m'] = (silvretta_temp['WFJ mean temperature (°C)'] - silvretta_temp['DAV mean temperature (°C)'])/1097
silvretta_temp['SVH expected mean temperature (°C)'] = silvretta_temp['difference/m'] * 799 + silvretta_temp['DAV mean temperature (°C)']

silvretta_temp = silvretta_temp[['SVH expected mean temperature (°C)']]
silvretta_temp

,SVH expected mean temperature (°C)
date,
1959-10-01,0.913765
1959-11-01,-4.293710
1959-12-01,-6.322060
1960-01-01,-9.077575
1960-02-01,-6.650410
...,...
2025-05-01,1.783045
2025-06-01,10.084230
2025-07-01,7.020055


In [4]:
silvretta_temp = weissfluhjoch_temp.merge(davos_temp, left_index=True, right_index=True)
silvretta_temp['difference/m'] = (silvretta_temp['WFJ mean temperature (°C)'] - silvretta_temp['DAV mean temperature (°C)'])/1097
silvretta_temp['SVH expected mean temperature (°C)'] = silvretta_temp['difference/m'] * 799 + silvretta_temp['DAV mean temperature (°C)']

silvretta_temp = silvretta_temp[['SVH expected mean temperature (°C)']]
silvretta_temp

silvretta_temp = silvretta_temp.reset_index(drop=True)

# Step 1: Add a helper column to group every 12 rows
silvretta_temp['year'] = silvretta_temp.index // 12

# Step 2: Add a helper column for the position within each group
silvretta_temp['month'] = silvretta_temp.index % 12

# Step 3: Pivot the DataFrame
silvretta_temp = silvretta_temp.pivot(index = 'year',
                                   columns = 'month',
                                   values = 'SVH expected mean temperature (°C)')

# Define the new column names
monthly_observations = [
    'october_t', 'november_t', 'december_t',
    'january_t', 'february_t', 'march_t',
    'april_t', 'may_t', 'june_t',
    'july_t', 'august_t', 'september_t'
]

# Rename the columns
silvretta_temp.columns = monthly_observations

silvretta_temp = silvretta_temp.reset_index(drop=True)
silvretta_temp = silvretta_temp.round(1)

silvretta_w_data = silvretta_temp.merge(silvrettahütte_prec, left_index=True, right_index=True)


# Create the new column with the start date of the hydrological year
silvretta_w_data['hydrological year'] = [
    f"{1959 + i}-{1960 + i}" for i in range(len(silvretta_w_data))
]


silvretta_w_data



,october_t,november_t,december_t,january_t,february_t,march_t,april_t,may_t,june_t,july_t,august_t,september_t,precipitation (mm),hydrological year
0,0.9,-4.3,-6.3,-9.1,-6.7,-5.3,-3.4,1.8,5.3,4.9,6.0,2.6,1160,1959-1960
1,-1.4,-3.6,-7.2,-7.5,-5.4,-4.6,-0.2,-0.6,5.4,5.3,6.8,8.0,1580,1960-1961
2,1.8,-3.5,-6.1,-7.3,-9.5,-9.9,-4.3,-0.7,3.0,5.8,8.6,4.5,1400,1961-1962
3,2.2,-6.1,-9.2,-13.1,-11.3,-6.1,-2.0,0.5,4.8,8.1,6.1,5.1,1180,1962-1963
4,1.5,-2.3,-7.6,-6.4,-7.2,-6.0,-2.1,2.7,6.3,8.1,5.9,5.1,1250,1963-1964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,-0.1,0.4,-5.8,-9.5,-3.8,-5.8,-4.6,-0.6,8.0,8.0,6.9,6.6,1507,2020-2021
62,1.3,-3.1,-5.0,-6.2,-6.1,-3.9,-1.9,4.4,8.8,10.0,9.5,3.7,1521,2021-2022
63,5.4,-2.1,-4.5,-7.3,-5.0,-3.5,-3.6,2.4,7.6,9.3,9.1,9.1,1182,2022-2023
64,4.6,-4.7,-4.1,-5.4,-3.1,-3.3,-1.4,2.1,6.4,10.0,11.5,3.8,1469,2023-2024


In [5]:
silvretta_mb = mass_balance_hy_df[(mass_balance_hy_df['glacier name'] == 'Silvrettagletscher')]

silvretta_mb = silvretta_mb[['start date of observation (yyyy-mm-dd)',
                                 'annual mass balance (mm w.e.)'
                            ]]
silvretta_mb = silvretta_mb.reset_index(drop=True)
silvretta_mb['start date of observation (yyyy-mm-dd)'] = pd.to_datetime(
    silvretta_mb['start date of observation (yyyy-mm-dd)']
)

cutoff = pd.to_datetime("1959-09-30")

silvretta_mb = silvretta_mb[
    silvretta_mb['start date of observation (yyyy-mm-dd)'] > cutoff
].reset_index(drop=True)
silvretta_mb = silvretta_mb[['annual mass balance (mm w.e.)']]
silvretta_mb

,annual mass balance (mm w.e.)
0,251
1,-388
2,32
3,-1070
4,-1474
...,...
61,-888
62,-3339
63,-2309
64,-1654


In [6]:
silvretta_model_df = silvretta_mb.merge(silvretta_w_data, left_index=True, right_index=True)
silvretta_model_df

,annual mass balance (mm w.e.),october_t,november_t,december_t,january_t,february_t,march_t,april_t,may_t,june_t,july_t,august_t,september_t,precipitation (mm),hydrological year
0,251,0.9,-4.3,-6.3,-9.1,-6.7,-5.3,-3.4,1.8,5.3,4.9,6.0,2.6,1160,1959-1960
1,-388,-1.4,-3.6,-7.2,-7.5,-5.4,-4.6,-0.2,-0.6,5.4,5.3,6.8,8.0,1580,1960-1961
2,32,1.8,-3.5,-6.1,-7.3,-9.5,-9.9,-4.3,-0.7,3.0,5.8,8.6,4.5,1400,1961-1962
3,-1070,2.2,-6.1,-9.2,-13.1,-11.3,-6.1,-2.0,0.5,4.8,8.1,6.1,5.1,1180,1962-1963
4,-1474,1.5,-2.3,-7.6,-6.4,-7.2,-6.0,-2.1,2.7,6.3,8.1,5.9,5.1,1250,1963-1964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,-888,-0.1,0.4,-5.8,-9.5,-3.8,-5.8,-4.6,-0.6,8.0,8.0,6.9,6.6,1507,2020-2021
62,-3339,1.3,-3.1,-5.0,-6.2,-6.1,-3.9,-1.9,4.4,8.8,10.0,9.5,3.7,1521,2021-2022
63,-2309,5.4,-2.1,-4.5,-7.3,-5.0,-3.5,-3.6,2.4,7.6,9.3,9.1,9.1,1182,2022-2023
64,-1654,4.6,-4.7,-4.1,-5.4,-3.1,-3.3,-1.4,2.1,6.4,10.0,11.5,3.8,1469,2023-2024


In [7]:
silvretta_model_df2 = silvretta_mb.merge(davos_t_dev, left_index=True, right_index=True)
silvretta_model_df2 = silvretta_model_df2.merge(silvrettahütte_prec, left_index=True, right_index=True)
silvretta_model_df2

,annual mass balance (mm w.e.),october_td,november_td,december_td,january_td,february_td,march_td,april_td,may_td,june_td,july_td,august_td,september_td,hydrological year,precipitation (mm)
0,251,-1.2,-2.4,-0.1,-1.1,0.9,0.4,-1.4,-0.1,-0.9,-3.5,-2.7,-2.0,1959-1960,1160
1,-388,-2.5,0.0,-1.1,-0.5,2.1,-0.3,1.6,-3.0,-0.7,-2.9,-1.9,2.7,1960-1961,1580
2,32,-0.2,-0.5,0.3,0.0,-2.2,-4.8,-2.6,-3.6,-2.8,-2.6,-0.6,-1.1,1961-1962,1400
3,-1070,-1.0,-2.8,-3.9,-5.9,-4.8,-2.0,-0.6,-2.2,-1.8,-0.8,-2.6,-0.5,1962-1963,1180
4,-1474,-1.3,0.8,-3.1,-0.9,0.6,-1.1,-0.3,0.3,-0.3,-0.6,-2.8,-0.5,1963-1964,1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,-888,-1.4,2.1,0.3,-1.9,2.6,-0.9,-2.1,-2.4,2.2,-0.3,-1.1,1.8,2020-2021,1507
62,-3339,-0.4,-1.0,0.6,0.6,0.8,0.7,0.3,2.3,2.5,2.0,1.7,-0.6,2021-2022,1521
63,-2309,3.5,1.3,1.1,0.4,2.0,1.6,-0.9,0.5,2.0,1.2,1.0,3.7,2022-2023,1182
64,-1654,3.1,-1.1,1.4,1.2,4.9,2.2,0.6,0.4,0.7,1.8,2.9,-0.3,2023-2024,1469


In [8]:
# Target
y = silvretta_model_df2['annual mass balance (mm w.e.)']

# Features: temperatures April–September + annual precipitation
feature_cols = ['april_td', 'may_td', 'june_td', 'july_td', 'august_td', 'september_td', 'precipitation (mm)']
X = silvretta_model_df2[feature_cols]

In [9]:
split = int(len(silvretta_model_df2) * 0.8)  # first 80% train, last 20% test

X_train, y_train = X.iloc[:split], y.iloc[:split]
X_test, y_test   = X.iloc[split:], y.iloc[split:]

In [10]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [11]:
y_pred = linreg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2   = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R²:", r2)

RMSE: 593.3442526765979
R²: 0.4220381785758701


In [12]:
coef_table = dict(zip(feature_cols, linreg.coef_))
print("Feature coefficients:")
for feature, coef in coef_table.items():
    print(f"{feature}: {coef:.4f}")

Feature coefficients:
april_td: -90.1901
may_td: -30.2282
june_td: -151.5409
july_td: -140.9311
august_td: -149.1261
september_td: -93.2406
precipitation (mm): 1.1459


In [14]:
davos_prec_pivotted = davos_prec.copy()


# Step 1: Add a helper column to group every 12 rows
davos_prec_pivotted['year'] = davos_prec_pivotted.index // 12

# Step 2: Add a helper column for the position within each group
davos_prec_pivotted['month'] = davos_prec_pivotted.index % 12

# Step 3: Pivot the DataFrame
davos_prec_pivotted = davos_prec_pivotted.pivot(index = 'year',
                                   columns = 'month',
                                   values = 'rhs150m0')

# Define the new column names
monthly_observations_prec = [
    'october_p', 'november_p', 'december_p',
    'january_p', 'february_p', 'march_p',
    'april_p', 'may_p', 'june_p',
    'july_p', 'august_p', 'september_p'
]


# Rename the columns
davos_prec_pivotted.columns = monthly_observations_prec

davos_prec_pivotted = davos_prec_pivotted.reset_index(drop=True)
davos_prec_pivotted = davos_prec_pivotted.round(1)

#silvretta_w_data = silvretta_temp.merge(silvrettahütte_prec, left_index=True, right_index=True)


# Create the new column with the start date of the hydrological year
#silvretta_w_data['hydrological year'] = [
#    f"{1959 + i}-{1960 + i}" for i in range(len(silvretta_w_data))
#]


davos_prec_pivotted

,october_p,november_p,december_p,january_p,february_p,march_p,april_p,may_p,june_p,july_p,august_p,september_p
0,73.9,42.1,21.7,101.3,40.8,82.2,35.4,88.0,86.4,192.9,72.8,107.6
1,5.4,46.2,52.2,116.8,106.3,68.3,57.8,43.1,212.9,118.6,135.5,85.2
2,103.7,83.9,114.8,69.4,7.6,22.6,81.9,60.9,72.5,130.6,194.5,46.5
3,138.4,42.3,32.1,31.9,35.7,36.3,51.5,18.2,118.0,80.1,110.5,130.3
4,49.5,23.6,227.2,41.6,40.5,31.5,91.4,61.4,125.2,138.6,65.5,57.5
...,...,...,...,...,...,...,...,...,...,...,...,...
106,110.2,6.5,75.8,158.4,24.2,50.4,30.3,91.2,48.2,182.8,157.2,54.4
107,12.1,95.8,51.6,35.5,85.6,5.0,33.9,67.4,131.1,92.7,85.8,114.4
108,98.1,42.8,39.1,18.3,23.0,71.9,85.2,112.5,76.4,118.8,218.6,69.6
109,105.7,137.1,166.1,66.5,48.9,114.7,51.0,115.4,100.5,136.0,85.1,142.0


In [20]:
silvretta_mb_full = mass_balance_hy_df[(mass_balance_hy_df['glacier name'] == 'Silvrettagletscher')]

silvretta_mb_full = silvretta_mb_full[['start date of observation (yyyy-mm-dd)',
                                 'annual mass balance (mm w.e.)'
                            ]]
silvretta_mb_full = silvretta_mb_full.reset_index(drop=True)
silvretta_mb_full['start date of observation (yyyy-mm-dd)'] = pd.to_datetime(
    silvretta_mb_full['start date of observation (yyyy-mm-dd)']
)

silvretta_mb_full = silvretta_mb_full[['annual mass balance (mm w.e.)']]
silvretta_mb_full

,annual mass balance (mm w.e.)
0,-415
1,381
2,-28
3,-414
4,1022
...,...
106,-888
107,-3339
108,-2309
109,-1654


In [22]:
silvretta_model_df3 = silvretta_mb_full.merge(davos_t_dev, left_index=True, right_index=True)
silvretta_model_df3 = silvretta_model_df3.merge(davos_prec_pivotted, left_index=True, right_index=True)
silvretta_model_df3

,annual mass balance (mm w.e.),october_td,november_td,december_td,january_td,february_td,march_td,april_td,may_td,june_td,...,december_p,january_p,february_p,march_p,april_p,may_p,june_p,july_p,august_p,september_p
0,-415,-1.9,-2.6,0.2,-2.4,-1.2,-2.4,-2.4,1.3,-0.1,...,21.7,101.3,40.8,82.2,35.4,88.0,86.4,192.9,72.8,107.6
1,381,-4.0,-4.2,2.3,0.1,-0.3,-0.4,-1.2,-0.7,-3.9,...,52.2,116.8,106.3,68.3,57.8,43.1,212.9,118.6,135.5,85.2
2,-28,-2.4,-1.7,-0.3,-3.9,-3.2,-3.1,-5.0,1.1,0.1,...,114.8,69.4,7.6,22.6,81.9,60.9,72.5,130.6,194.5,46.5
3,-414,-4.5,-3.2,-5.1,-0.8,-1.1,-1.4,-0.5,0.1,-4.3,...,32.1,31.9,35.7,36.3,51.5,18.2,118.0,80.1,110.5,130.3
4,1022,-2.8,-1.2,0.2,-2.2,-2.4,-1.9,-3.8,-3.1,-2.1,...,227.2,41.6,40.5,31.5,91.4,61.4,125.2,138.6,65.5,57.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-888,-1.4,2.1,0.3,-1.9,2.6,-0.9,-2.1,-2.4,2.2,...,75.8,158.4,24.2,50.4,30.3,91.2,48.2,182.8,157.2,54.4
107,-3339,-0.4,-1.0,0.6,0.6,0.8,0.7,0.3,2.3,2.5,...,51.6,35.5,85.6,5.0,33.9,67.4,131.1,92.7,85.8,114.4
108,-2309,3.5,1.3,1.1,0.4,2.0,1.6,-0.9,0.5,2.0,...,39.1,18.3,23.0,71.9,85.2,112.5,76.4,118.8,218.6,69.6
109,-1654,3.1,-1.1,1.4,1.2,4.9,2.2,0.6,0.4,0.7,...,166.1,66.5,48.9,114.7,51.0,115.4,100.5,136.0,85.1,142.0


In [23]:
df = silvretta_model_df3.copy()

y = df.iloc[:, 0]

precip_cols = [c for c in df.columns if c.endswith('_p')]
temp_cols = ["april_td", "may_td", "june_td", "july_td",
             "august_td", "september_td"]

X = df[precip_cols + temp_cols]

reg = LinearRegression().fit(X, y)

print("R^2:", reg.score(X, y))
print("Intercept:", reg.intercept_)
print("Coefficients:", dict(zip(X.columns, reg.coef_)))

R^2: 0.7795024924057964
Intercept: -3379.2218373512346
Coefficients: {'october_p': np.float64(3.0108002069478217), 'november_p': np.float64(2.476666085757776), 'december_p': np.float64(3.452341151662627), 'january_p': np.float64(1.913612186269839), 'february_p': np.float64(3.0892034317243584), 'march_p': np.float64(1.7299447706657343), 'april_p': np.float64(2.009832955640043), 'may_p': np.float64(2.7835447314821034), 'june_p': np.float64(-0.21204405768262502), 'july_p': np.float64(2.3144145830688627), 'august_p': np.float64(3.7754454415237193), 'september_p': np.float64(1.963081857840993), 'april_td': np.float64(-1.6919804854207807), 'may_td': np.float64(-72.24652694724362), 'june_td': np.float64(-186.03596816503978), 'july_td': np.float64(-187.97196316935563), 'august_td': np.float64(-95.37183549980547), 'september_td': np.float64(-129.16211054458313)}
